In [ ]:
!pip install keras_tuner

In [ ]:
import os
import re
import pickle
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from keras.preprocessing import text, sequence
from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='loss', patience=10)#, verbose=1)
from sklearn.preprocessing import LabelEncoder
# keras Tuner
import keras_tuner as kt
# HP
from keras_tuner import Hyperband

from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
import nltk
import string 
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
lb_enc = LabelEncoder()
from keras_tuner import RandomSearch

In [ ]:
data_train = pd.read_excel('/kaggle/input/spam-review-detection/data_train.xlsx')
data_dev = pd.read_excel('/kaggle/input/spam-review-detection/data_dev.xlsx')
data_test = pd.read_excel('/kaggle/input/spam-review-detection/data_test.xlsx')

In [ ]:
y_train=data_train['Label']

y_dev=data_dev['Label']

y_test=data_test['Label']

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(data_train['transformed_text'])
tokenizer.fit_on_texts(data_dev['transformed_text'])
tokenizer.fit_on_texts(data_test['transformed_text'])

text_to_sequence_train = tokenizer.texts_to_sequences(data_train['transformed_text']) 
text_to_sequence_dev = tokenizer.texts_to_sequences(data_dev['transformed_text']) 
text_to_sequence_test = tokenizer.texts_to_sequences(data_test['transformed_text']) 

In [ ]:
max_length_sequence_train = max([len(i) for i in text_to_sequence_train])
 
padded_train = pad_sequences(text_to_sequence_train, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
max_length_sequence_train
padded_train

In [ ]:
padded_dev = pad_sequences(text_to_sequence_dev, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_dev)

In [ ]:
padded_test = pad_sequences(text_to_sequence_test, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_test)

In [ ]:
VOC_SIZE = len(tokenizer.word_index)+1 # 18359

def build_model(hp):
    model = Sequential()
    model.add(Embedding(VOC_SIZE, hp.Int('embedding_dim',  min_value=200, max_value=400, step=50), input_length=max_length_sequence_train))
    
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(GRU(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), 
                       activation='tanh', recurrent_activation='sigmoid', 
                       return_sequences=(i < (hp.Int('num_layers', 1, 3) - 1))))
        model.add(Dropout(hp.Float('dropout_' + str(i), 0.4, 0.9, 0.1)))
    
    model.add(Dense(units=hp.Int('dense_units', 32, 256, 32), activation=hp.Choice('dense_activation', ['relu', 'tanh', 'sigmoid', 'elu'])))
    model.add(Dropout(hp.Float('dense_dropout', 0.4, 0.9, 0.1)))
    
    model.add(Dense(units=1, activation=hp.Choice('dense_activation', ['relu', 'tanh', 'sigmoid', 'elu'])))
    
    # Epochs
    num_epochs = hp.Int('num_epochs', min_value=10, max_value=100, step=10)
    # batch_size
    batch_size=hp.Int('batch_size', 16, 128, step=16)
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 0.0001, 0.01, step=0.0001)), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model